In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

DATA WRANGLING


CHECK FOR THE ROWS WITH MISSING WIN PREDICTION

In [ ]:
df['winPlacePerc'].isnull()

In [ ]:
# check row with null value win place perc
df[df['winPlacePerc'].isnull()]

In [ ]:
# removing that row
df.drop(2744604, inplace = True)

In [ ]:
df.dropna()

In [ ]:
df.shape

Data Descritption
DBNOs - Number of enemy players knocked.
assists - Number of enemy players this player damaged that were killed by teammates.
boosts - Number of boost items used.
damageDealt - Total damage dealt. Note: Self inflicted damage is subtracted.
headshotKills - Number of enemy players killed with headshots.
heals - Number of healing items used.
Id - Player’s Id
killPlace - Ranking in match of number of enemy players killed.
killPoints - Kills-based external ranking of player. (Think of this as an Elo ranking where only kills matter.) If there is a value other than -1 in rankPoints, then any 0 in killPoints should be treated as a “None”.
killStreaks - Max number of enemy players killed in a short amount of time.
kills - Number of enemy players killed.
longestKill - Longest distance between player and player killed at time of death. This may be misleading, as downing a player and driving away may lead to a large longestKill stat.
matchDuration - Duration of match in seconds.
matchId - ID to identify match. There are no matches that are in both the training and testing set.
matchType - String identifying the game mode that the data comes from. The standard modes are “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, and “squad-fpp”; other modes are from events or custom matches.
rankPoints - Elo-like ranking of player. This ranking is inconsistent and is being deprecated in the API’s next version, so use with caution. Value of -1 takes place of “None”.
revives - Number of times this player revived teammates.
rideDistance - Total distance traveled in vehicles measured in meters.
roadKills - Number of kills while in a vehicle.
swimDistance - Total distance traveled by swimming measured in meters.
teamKills - Number of times this player killed a teammate.
vehicleDestroys - Number of vehicles destroyed.
walkDistance - Total distance traveled on foot measured in meters.-
weaponsAcquired - Number of weapons picked up.
winPoints - Win-based external ranking of player. (Think of this as an Elo ranking where only winning matters.) If there is a value other than -1 in rankPoints, then any 0 in winPoints should be treated as a “None”.
groupId - ID to identify a group within a match. If the same group of players plays in different matches, they will have a different groupId each time.
numGroups - Number of groups we have data for in the match.
maxPlace - Worst placement we have data for in the match. This may not match with numGroups, as sometimes the data skips over placements.
winPlacePerc - The target of prediction. This is a percentile winning placement, where 1 corresponds to 1st place, and 0 corresponds to last place in the match. It is calculated off of maxPlace, not numGroups, so it is possible to have missing chunks in a match.

Understanding the Player distribution

In [ ]:
df['playerjoined']=df.groupby('matchId')['matchId'].transform('count')
df.head()

Analysing the data

Kill Without Moving¶

In [ ]:
# prepare the parameter to gather the information of tha total distance travelled
df['totaldistance']=df['rideDistance']+ df['swimDistance']+ df['walkDistance']


# detecting the person does not move but managed to kill
df['killwithoutmoving']=((df['kills']>0) & (df['totaldistance']==0))

In [ ]:
df[df['killwithoutmoving']==True]

players who travel zero distance yet they have killed enemies seems suspicious, hence removing those rows!!¶

In [ ]:
df[df['killwithoutmoving']==True].shape

In [ ]:
# drop the table
df.drop(df[df['killwithoutmoving']== True].index , inplace = True)

In [ ]:
df.shape

Extra Ordinary Road Kills

In [ ]:
df[df['roadKills']>=5].shape

In [ ]:
df.drop(df[df['roadKills']>=5].index , inplace = True)

In [ ]:
df[df['roadKills']>=5].shape

In a match a player cannot not kill more that 20 kill

In [ ]:
df[df['kills']>20].shape

In [ ]:
# removing these raw
df.drop(df[df['kills']>20].index , inplace = True)

HEAD SHOT

In [ ]:
df['headshot_rate'] = df['headshotKills']/df['kills']
df['headshot_rate'] = df['headshot_rate'].fillna(0)

In [ ]:
df.head()

In [ ]:
# plotting the headshot rate distribution
sns.distplot(df['headshot_rate'])
plt.ylabel('Count of Players')
plt.show()

In [ ]:
df[(df['headshot_rate']==1) & (df['kills']>5)].shape

In [ ]:
# removing the row because its rarely possible that a person has more than 5 kills and all of them are head shot
df.drop(df[(df['headshot_rate'] == 1)& (df['kills'] > 5)].index, inplace = True)

LONGEST SHOT


In [ ]:
sns.distplot(df['longestKill'],bins = 50)
plt.ylabel("count of player")
plt.show()

In [ ]:
df[df['longestKill']>500].shape

In [ ]:
# if a player can kill more than 500m distance it seem suspisious player so we remove those player
df.drop(df[df['longestKill']>500].index, inplace = True)

WEAPON CHANGE

In [ ]:
sns.distplot(df['weaponsAcquired'],bins = 10)
plt.show()

In [ ]:
df[df['weaponsAcquired']>=15].shape

In [ ]:
df.drop(df[df['weaponsAcquired']>=15].index, inplace = True)

correlation of parameter with Win Prediction

In [ ]:
plt.figure(figsize=[15,15])
sns.heatmap(df.corr(), annot = True)
plt.show()

FEATURE ENGINEERING


In [ ]:
normalising_factor = (100 - df['playerjoined']/100)+1

In [ ]:
df['killsNorm']= df['kills']*normalising_factor
df['damageDealtNorm'] = df['damageDealt'] * normalising_factor
df['maxPlaceNorm'] = df['maxPlace'] * normalising_factor
df['matchDurationNorm'] = df['matchDuration'] * normalising_factor


df['traveldistance'] = df['walkDistance'] + df['swimDistance'] + df['rideDistance']
df['healsnboosts'] = df['heals'] + df['boosts']
df['assist'] = df['assists'] + df['revives']

In [ ]:
df.columns

In [ ]:
data = df.drop(columns=['Id', 'groupId', 'matchId', 'assists', 'boosts', 'walkDistance', 'swimDistance', 
                          'rideDistance', 'heals', 'revives', 'kills', 'damageDealt', 'maxPlace', 'matchDuration'])

In [ ]:
data.head()

In [ ]:
# HANDELING CATEGORICAL DATA

In [ ]:
data = pd.get_dummies(data,columns=['matchType','killwithoutmoving'])

In [ ]:
data.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
x=data.drop(['winPlacePerc'],axis=1)
y=data['winPlacePerc']

In [ ]:
feature = x.columns

SCALLING THE DATA

In [ ]:
sc = StandardScaler()
sc.fit(x)
x = pd.DataFrame(sc.transform(x))

In [ ]:
x.head()

SPLITTING THE DATA

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y, test_size = 0.3,random_state=0)

In [ ]:
print(xtrain.shape,ytrain.shape)
print(xtest.shape,ytest.shape)

Using Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
parameters = {
   'n_estimators':[110,130,140,150,160,180], 
   'max_depth': np.arange(2, 17, 2)
}
rf = RandomForestRegressor()
grid_search = GridSearchCV(estimator = rf, param_grid = parameters, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [ ]:
grid_search.fit(xtrain, ytrain)

In [ ]:
y_pred = grid_search.predict(x_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(ytest,y_pred ))
r2 = r2_score(ytest, y_pred)

print("Testing performance")

print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

CATBOOST MODEL

In [ ]:
import catboost as cb

In [ ]:
train_dataset = cb.Pool(xtrain, ytrain)
test_dataset = cb.Pool(xtest, ytest)

In [ ]:
'''
grid = {'iterations': [100, 150], 
       'learning_rate': [0.03, 0.1], 
       'depth': [2, 4, 6,8]}

model.grid_search(grid, train_dataset)
'''

In [ ]:
model = cb.CatBoostRegressor(iterations=150,learning_rate=0.1,depth=8,loss_function='RMSE')

In [ ]:
model.fit(train_dataset)

In [ ]:
pred = model.predict(xtest)

In [ ]:
rmse = np.sqrt(mean_squared_error(ytest, pred))
r2 = r2_score(ytest, pred)

print("Testing performance")

print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

In [ ]:
feature_importance_df = pd.DataFrame()
feature_importance_df['features'] = features
feature_importance_df['importance'] = model.feature_importances_

feature_importance_df = feature_importance_df.sort_values(by = ['importance'], ascending=False)

In [ ]:
feature_importance_df

In [ ]:
plt.bar(feature_importance_df.features, feature_importance_df.importance, color='turquoise')
plt.ylabel("CatBoost Feature Importance")
plt.xticks(rotation = 90)
plt.show()